In [1]:
import re
import os
import pandas as pd
import numpy as np
import traceback2
from datetime import datetime
import pymysql
import numpy as np

where_map = {}
# pairs regexp: "\[([a-zA-z0-9$\-+τ\.%]*)\]-", "-\[([a-zA-z0-9$\-+τ]*)\]"
ignorable_files = ["farm_risk_yield_data_2021_06_14.csv", "Farm_risk_06_14.csv", "yield_data_2021_06_14.csv","Alternate.csv","vfat_tools_data_2021_06_14.csv",'.ipynb_checkpoints', "Manual Farm Yield 6:14.xlsx", "crash_parser.ipynb", "yieldFarm_6-14.csv"]


In [2]:
def __connect__():
    db_user = 'coin'
    db_password = 'dragonstone#123'
    db_name = 'master_data'
    db_host = 'localhost'
    return pymysql.connect(
        unix_socket=None,
        host=db_host,
        user=db_user,
        password=db_password,
        db=db_name,
        charset='latin1',
        cursorclass=pymysql.cursors.DictCursor)


def get_cursor(connection):
    if connection is None:
        connection = __connect__()
    connection.ping(reconnect=True)
    return connection.cursor()

In [3]:
def get_raydium_data(date):
    sql = f'select * from raydium_farm_pool_data where timestamp_dt = FROM_UNIXTIME({date});'
    data = pd.read_sql_query(sql, __connect__())
    data['apr'] = data['pool_rate']
    data['network'] = 'SOL'
    data['type_of_agriculture'] = data['farm_payment'].apply(lambda x : 'FARM' if x is not None else 'POOL')
    return data[['pair1', 'pair2', 'apr', 'tvl', 'where', 'network', 'type_of_agriculture']]

def get_liquidityfolio_pool_data(date):
    sql = f'select * from liquidityfolio_pool where timestamp_dt = FROM_UNIXTIME({date}) and `where` != "balancer";'
    data = pd.read_sql_query(sql, __connect__())
    data['apr'] = data['pool_rate'] * 100 * 12
    data['type_of_agriculture'] = data['farm_payment'].apply(lambda x : 'FARM' if x is not None else 'POOL')
    data['network'] = 'ETH'
    return data[['pair1', 'pair2', 'apr', 'tvl', 'where', 'network', 'type_of_agriculture']]

In [4]:
date = datetime(2021, 6, 16)
dt = date.timestamp()

In [5]:
df = get_raydium_data(dt)
df = df.append(get_liquidityfolio_pool_data(dt))
df[df['where'] == 'Sushi']

,pair1,pair2,apr,tvl,where,network,type_of_agriculture
0,ETH,UMA,5.984138,3667.577108,Sushi,ETH,POOL
1,ETH,UWL,2.463383,53.551732,Sushi,ETH,POOL
4,ETH,LIGHT,0.206812,1.948623,Sushi,ETH,POOL
9,DGCL,ACXT,7.781073,11.161577,Sushi,ETH,POOL
11,UNI,xSUSHI,5.109741,0.011464,Sushi,ETH,POOL
...,...,...,...,...,...,...,...
1442,ETH,AMALGAM,0.087070,0.172533,Sushi,ETH,POOL
1443,USDP,DUCK,2.729615,94.412030,Sushi,ETH,POOL
1444,DAI,RDM,0.000000,1.513494,Sushi,ETH,POOL
1449,ETH,89SA,5.153667,3.998762,Sushi,ETH,POOL


In [6]:
sm = {
    "WBTC" : "BTC",
    "WETH" : "ETH",
    "WBNB" : "BNB",
    "WFTM" : "FTM",
    "WAVAX" : "AVAX",
}


In [7]:
dex_map = {
    "SUSHI" : "https://app.sushi.com"
    
}

In [8]:
def pair1(line):
#     print(line)
    try:
        is_3_pairs = line.split("]-[")
        if len(is_3_pairs) > 2:
            return None
        s_index = line.index("[") + 1
        s = line[s_index: line.index("]", s_index)]
        if all(ord(c) < 128 for c in s):
            return s
        else:
            print(line)
            return None
    except:
        return ''

def pair2(line):
#     print(line)
    s_index = line.index("-[") + 2
    s = line[s_index: line.index("]", s_index)]
#     print(s)
    if all(ord(c) < 128 for c in s):
        return s
    else:
        print(line)
        return ''

In [9]:
def apr(line):
    apr = "Year [0-9\,\.]+\%"
    x = re.findall(apr, line)
    return x[0].replace("Year ", "").replace("%", "").strip() if len(x) else 0

In [10]:
def tvl(line):
    tvl = "TVL: \$[0-9\.\,]+"
    x = re.findall(tvl, line)
    return x[0].replace("TVL: $", "").replace("%", "").replace(",","").strip()

In [11]:
def where(line):
    if 'INFO' in line:
        return line.replace("INFO", "").replace('\n', '').replace(":", "").replace("//", "://").replace('*', '').strip()
    return None

In [12]:
def get_single_pair(line):
    exp = "\d* *.*Price:"
    x = re.findall(exp, line)
    if not len(x) or "TVL:" in line:
#         print(">>>>>>>>>>>",line)
        return
    pair = x[0].replace(" Price:", "")
    return pair[pair.rindex(" "):].strip() if " " in pair else pair

def get_single_tvl(line):
    exp = "\([0-9\,\.$]+\)"
    x = re.findall(exp, line)
    return x[0].replace("(", "").replace(")", "").replace("$", "").replace(",","").strip()


In [13]:
tvl("[WBTC]-[WAVAX] PGL [+] [-] [<=>] Price: $160,706,376.77 TVL: $12,218,373.64\n")

'12218373.64'

In [14]:
def is_end_of_block(line):
#     print(line, line == "" or len(re.findall("INFO[ ]+: ", line)))
    return line.strip() == "" or len(re.findall("INFO[ ]+: ", line)) or "******" in line

def read_block(lines, start_index):
    block = []
#     print("Block Start Line No.: ", start_index + 1)
    while start_index < len(lines):
        line = lines[start_index]
        start_index = start_index + 1
        if not is_end_of_block(line):
            block.append(line.strip())
        else:
            break
    return [block, start_index]

def read_blocks_from_files(file):
    f = open(file)
    lines = f.readlines()
    blocks = []
    block = []
    block_start_line_no = []
    line_no = 0
    start_index = 0
    while start_index < len(lines):
        line = lines[start_index]
        if not is_end_of_block(lines[start_index]):
            block_and_index = read_block(lines, start_index)
#             print(block_and_index[0])
            if block_and_index is not None:
                block_start_line_no.append(start_index + 1)
                start_index = block_and_index[1]
                blocks.append(block_and_index[0])
            else:
                start_index = start_index + 1
        elif len(re.findall("INFO[ ]+: ", line)):
            block_start_line_no.append(start_index + 1)
            blocks.append([line])
            start_index = start_index + 1
        else:
            start_index = start_index + 1
    return [blocks, block_start_line_no]

def read_blocks_from_files_as(file):
    f = open(file)
    lines = f.readlines()
    blocks = []
    block = []
    block_start_line_no = []
    line_no = 0
    for line in lines:
        line_no = line_no + 1
        line = line.strip()
        if line == "" or len(re.findall("INFO[ ]+: ", line)):
            blocks.append(block)
            block_start_line_no.append(line_no)
            if len(line):
                blocks.append([line])
                block_start_line_no.append(line_no + 1)
            block = []
        else:
            block.append(line)
            
    blocks.append(block)
    return [blocks, block_start_line_no]

def get_pair_info_from_block(block, wh, network):
    tvl_index = 0
    if network == 'BSC':
        tvl_index = 3
    if len(block) == 1:
        if 'INFO' in block[0]:
            return [where(block[0])]
        else:
            return [wh] if len(wh) else None
    if len(block) and pair1(block[0]) and not get_single_pair(block[0]):
        apr_index = -1
        for index in range(1, len(block)):
            if 'APR: ' in block[index]:
                apr_index = index
                break
        return [pair1(block[0]), pair2(block[0]), get_single_tvl(block[tvl_index]) if tvl_index > 0 else tvl(block[tvl_index]), apr(block[apr_index]) if apr_index > 1 else 0, where_map[wh] if wh in where_map else wh, network]
    elif len(block) > 2 and get_single_pair(block[0]):
#         print('singel')
        if network == 'BSC':
            tvl_index = 1
        apr_index = -1
        for index in range(1, len(block)):
            if 'APR: ' in block[index]:
                apr_index = index
                break
        return [get_single_pair(block[0]), "", get_single_tvl(block[1]), apr(block[apr_index]) if apr_index > 1 else 0, where_map[wh] if wh in where_map else wh, network]
    return None


def get_pairs(blocks, network, block_start_line_no):
    where = ""
    pairs = []
    index = -1
    for block in blocks:
#         print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        index = index + 1
        try:
            pair = get_pair_info_from_block(block, where, network)
            if pair == None:
                print('no Pair....\n',block)
                print("Block Start Line No.: ", block_start_line_no[index] if index < len(block_start_line_no) else -1)
#             if len(where) == 0 and pair and len(pair) > 2:
#                 print(block, network)
            if pair is not None and len(pair) > 0:
                if len(pair) == 1 and pair[0] and len(pair[0]):
                    where = pair[0]
                else:
                    pairs.append(pair)
        except Exception as e:
            print("Exception", traceback2.format_exc())
            print(block[0])
    return pairs
        

In [15]:
block = [['[WBTC]-[WAVAX] PGL [+] [-] [<=>] Price: $160,706,376.77 TVL: $12,218,373.64\n', 'WBTC Price: $40,623.32\n', 'WAVAX Price: $15.08\n', 'Staked: 0.0757 PGL ($12,159,145.14)\n', 'PNG Per Week: 53469.64 ($72,184.01)\n', 'APR: Day 0.08% Week 0.59% Year 30.87%\n', 'You are staking 0.000000 [WBTC]-[WAVAX] PGL $0.00 (0.00% of the pool).\n', 'Explorer\n', 'Stake 0.000000 [WBTC]-[WAVAX] PGL\n', 'Unstake 0.000000 [WBTC]-[WAVAX] PGL\n', 'Claim 0.000000 PNG ($0.00)\n', 'Revoke (set approval to 0)\n', 'Exit\n']]
get_pairs(block, "AVAX", [1])

[['WBTC', 'WAVAX', '12218373.64', '30.87', '', 'AVAX']]

In [16]:
data_dir = './'
def conver_files_to_one_df(data_dir):
    all_pairs = []
    files = os.listdir(data_dir)
    for file_name in files:
        if file_name in ignorable_files:
            continue
        network = file_name[0:file_name.index(" ")]
        if 'Alternate' == network:
            network = 'ETH'
#         if 'MATIC' != network:
#             continue
        print(file_name)
        blocks = read_blocks_from_files(f"{data_dir}{file_name}")
        block_start_line_no = blocks[1]
        blocks = blocks[0]
        pairs = get_pairs(blocks, network, block_start_line_no)
        all_pairs.extend(pairs)
    return all_pairs

In [17]:
pairs = conver_files_to_one_df(data_dir)

Alternate Format Yields 6:14.crash
no Pair....
 ['Alternate Format Yields 6/14', '____________________________________________________________________']
Block Start Line No.:  1
no Pair....
 ['ETH', '____________________________________________________________________']
Block Start Line No.:  4
no Pair....
 ['Staking Contract', 'Found 278 pools.']
Block Start Line No.:  18
no Pair....
 ['|  #  |         Pair         | Total Staked | Total $ Staked  | Reward |   APR   |   MyStake', '|-----|----------------------|--------------|-----------------|--------|---------|---------------------|', '|   0 | [WETH]-[USDT] SLP    |       1.8143 | $254,933,857.26 | SUSHI  |  16.06% | 0.00 ($0.00), 0.00% |', '|   1 | [USDC]-[WETH] SLP    |       3.0352 | $432,904,080.49 | SUSHI  |  11.82% | 0.00 ($0.00), 0.00% |', '|   2 | [DAI]-[WETH] SLP     | 1759687.2668 | $232,082,030.74 | SUSHI  |  19.56% | 0.00 ($0.00), 0.00% |', '|   3 | [sUSD]-[WETH] SLP    |   75310.3916 |  $11,246,444.30 | SUSHI  |  19.08% 

no Pair....
 ['BSC Yields 6/14']
Block Start Line No.:  1
361 - [τBTC]-[BTCB] Cake LP [+] [-] [<=>] Price: $2,461,761,461.98 TVL: $22,742,822.56
no Pair....
 ['361 - [τBTC]-[BTCB] Cake LP [+] [-] [<=>] Price: $2,461,761,461.98 TVL: $22,742,822.56', 'τBTC Price: $36,953.19', 'BTCB Price: $40,490.00', 'Staked: 0.0091 Cake-LP ($22,380,295.12)', 'CAKE Per Week: 11004.37 ($191,365.94)', 'APR: Day 0.12% Week 0.86% Year 44.46%', 'You are staking 0.00 [τBTC]-[BTCB] Cake LP ($0.00), 0.00% of the pool.', 'Stake 0.00 [τBTC]-[BTCB] Cake LP', 'Unstake 0.00 [τBTC]-[BTCB] Cake LP', 'Claim 0.00 CAKE ($0.00)', 'Staking or unstaking also claims rewards.']
Block Start Line No.:  1165
380 - [WBNB]-[τDOGE] Cake LP [+] [-] [<=>] Price: $2,507,912.43 TVL: $4,547,514.05
334 - [τBTC]-[BTCB] Cake LP [+] [-] [<=>] Price: $2,422,571,476.53 TVL: $22,380,768.44
no Pair....
 ['334 - [τBTC]-[BTCB] Cake LP [+] [-] [<=>] Price: $2,422,571,476.53 TVL: $22,380,768.44', 'τBTC Price: $36,305.92', 'BTCB Price: $39,910.00', 

In [18]:
pair_df = pd.DataFrame(pairs, columns=['pair1', 'pair2', 'tvl', 'apr', 'where', 'network'])
pair_df['apy'] = np.nan

In [19]:
pairs

[['UNI', '', '58975.79', '133.38', 'https://penguinswap.eth.link', 'ETH'],
 ['BTCx5-USDC-1Jun21-LP',
  '',
  '852291.43',
  '437.23',
  'https://app.compli.fi',
  'ETH'],
 ['BTCx5-USDC-1Jun21-UP',
  '',
  '17926.99',
  '10393.52',
  'https://app.compli.fi',
  'ETH'],
 ['BTCx5-USDC-1Jun21-DOWN',
  '',
  '54131.76',
  '3442.06',
  'https://app.compli.fi',
  'ETH'],
 ['ETHx5-USDC-1Jun21-LP',
  '',
  '3847.21',
  '96862.25',
  'https://app.compli.fi',
  'ETH'],
 ['ETHx5-USDC-1Jun21-UP',
  '',
  '16401.84',
  '11359.98',
  'https://app.compli.fi',
  'ETH'],
 ['ETHx5-USDC-1Jun21-DOWN',
  '',
  '9466.05',
  '19683.45',
  'https://app.compli.fi',
  'ETH'],
 ['LINKx5-USDC-1Jun21-LP', '', '0.00', 0, 'https://app.compli.fi', 'ETH'],
 ['LINKx5-USDC-1Jun21-UP',
  '',
  '3305.44',
  '56369.16',
  'https://app.compli.fi',
  'ETH'],
 ['LINKx5-USDC-1Jun21-DOWN', '', '0.00', 0, 'https://app.compli.fi', 'ETH'],
 ['UNIx5-USDC-1Jun21-LP', '', '0.00', 0, 'https://app.compli.fi', 'ETH'],
 ['UNIx5-USDC-1Jun21

In [20]:
pair_df = pair_df.drop(pair_df[pair_df['pair1'] == ''].index)

In [21]:
pair_df[pair_df['pair1'] == '']

,pair1,pair2,tvl,apr,where,network,apy


In [22]:
pair_df[pair_df['network'] == 'Alternate']['network'] = 'ETH'

In [23]:
sheet_pairs_df = pd.read_csv('yieldFarm_6-14.csv')
eth_sheet_pairs_df = pd.read_csv('Alternate.csv')
# eth_sheet_pairs_df.columns=['pair1', 'pair2', 'tvl', 'where', 'apr']
eth_sheet_pairs_df['apy'] = np.nan
eth_sheet_pairs_df['network'] = 'ETH'
sheet_pairs_df = sheet_pairs_df.append(eth_sheet_pairs_df)
sheet_pairs_df['pair2'] = sheet_pairs_df['pair2'].fillna('')

In [24]:
sheet_pairs_df

,pair1,pair2,apr,apy,tvl,where,network
0,MEATBALL,BUSD,"3,998.26",NaN,150,https://swedefi.com/,BSC
1,MEATBALL,BNB,"3,381.76",NaN,156,https://swedefi.com/,BSC
2,BNB,BUSD,54.58,NaN,"1,378",https://swedefi.com/,BSC
3,CAKE,BNB,127.92,NaN,588,https://swedefi.com/,BSC
4,USDT,BUSD,103.82,NaN,290,https://swedefi.com/,BSC
...,...,...,...,...,...,...,...
200,BOND,WETH,104.65,NaN,"78,859",SUSHI,ETH
201,bb_cDAI,DAI,59.45,NaN,"138,810",SUSHI,ETH
202,WETH,FOLD,90.43,NaN,"91,256",SUSHI,ETH
203,NDX,WETH,55.22,NaN,"149,455",SUSHI,ETH


In [25]:
def update_apr(apr):
    apr = f"{apr}".replace("%", "")
    if len(apr) == 0 or "nan" == apr:
        return np.nan
    else:
        return apr

In [26]:
def get_symbol_mapping(pair):
    if pair is not None:
        return sm[pair.strip()] if pair.strip() in sm else pair.strip()
    else:
        return ""

In [27]:
pair_df = pair_df.append(sheet_pairs_df).reset_index(drop=True)
pair_df['type_of_agriculture'] = 'FARM'
pair_df = pair_df.append(df).reset_index(drop=True)

In [28]:

pair_df['apr'] = pair_df['apr'].apply(lambda x: update_apr(x))
pair_df['pair1'] = pair_df['pair1'].apply(lambda x: sm[x.strip()] if x.strip() in sm else x.strip())
pair_df['pair2'] = pair_df['pair2'].apply(lambda x: get_symbol_mapping(x))

In [29]:
pair_df['pair2'].unique()

array(['', 'ETH', 'QUICK', ..., 'CHZ', 'POLS', '89SA'], dtype=object)

In [30]:
pair_df

,pair1,pair2,tvl,apr,where,network,apy,type_of_agriculture
0,UNI,,58975.79,133.38,https://penguinswap.eth.link,ETH,NaN,FARM
1,BTCx5-USDC-1Jun21-LP,,852291.43,437.23,https://app.compli.fi,ETH,NaN,FARM
2,BTCx5-USDC-1Jun21-UP,,17926.99,10393.52,https://app.compli.fi,ETH,NaN,FARM
3,BTCx5-USDC-1Jun21-DOWN,,54131.76,3442.06,https://app.compli.fi,ETH,NaN,FARM
4,ETHx5-USDC-1Jun21-LP,,3847.21,96862.25,https://app.compli.fi,ETH,NaN,FARM
...,...,...,...,...,...,...,...,...
8290,ETH,0xMR,6.393288,24.30591528,Uniswap,ETH,NaN,POOL
8291,ETH,CHZ,40.560723,56.11707960000001,Uniswap,ETH,NaN,POOL
8292,ETH,POLS,4868.881122,4.3867717200000005,Uniswap,ETH,NaN,POOL
8293,ETH,89SA,3.998762,5.15366652,Sushi,ETH,NaN,POOL


In [31]:
# pair_df['apr'].unique().tolist()

In [32]:
def convert_apr_to_apy(pair_df):
    tmp_df = pair_df[pair_df['apy'].isna()]
    print(tmp_df.shape)
    for index, row in tmp_df.iterrows():
        apr = row['apr']
        if pd.isnull( apr):
            return
        if apr is not None and len(apr.strip()):
            apr = float(apr.replace(',',''))
        else:
            apr = 0
        try:
            pair_df.loc[index, 'apy'] = ((1+(apr/100)/365)**365-1)*100
        except:
            pair_df.loc[index, 'apy'] = np.nan
    return
            
def convert_apy_to_apr(pair_df):
    tmp_df = pair_df[(pair_df['apr'].isnull())]
    print(tmp_df.shape)
    for index, row in tmp_df.iterrows():
        apy = row['apy']
        if pd.isnull( apy):
            return
        if apy is not None and len(apy.strip()):
            apy = float(apy.replace(',',''))
        else:
            apy = 0
        try:
            pair_df.loc[index, 'apr'] = (((1+(apy/100))**(1/365)-1)*365)*100
        except:
            pair_df.loc[index, 'apr'] = np.nan
    return


convert_apr_to_apy(pair_df)
convert_apy_to_apr(pair_df)

(8295, 8)
(55, 8)


In [33]:
pair_df[pair_df['apy'].isna()]

,pair1,pair2,tvl,apr,where,network,apy,type_of_agriculture
593,Snake,,0.00,2782742.87,https://www.polysnake.finance,MATIC,NaN,FARM
800,USDC,POLLEN,9.19,348215.92,https://polybutterfly.finance,MATIC,NaN,FARM
806,USDC,,0.00,871232357.16,https://polybutterfly.finance,MATIC,NaN,FARM
808,QUICK,,0.01,3541396.63,https://polybutterfly.finance,MATIC,NaN,FARM
945,WMATIC,MEME,15343.17,3671762.35,https://berserkfinance.com,MATIC,NaN,FARM
...,...,...,...,...,...,...,...,...
8290,ETH,0xMR,6.393288,24.30591528,Uniswap,ETH,NaN,POOL
8291,ETH,CHZ,40.560723,56.11707960000001,Uniswap,ETH,NaN,POOL
8292,ETH,POLS,4868.881122,4.3867717200000005,Uniswap,ETH,NaN,POOL
8293,ETH,89SA,3.998762,5.15366652,Sushi,ETH,NaN,POOL


In [34]:
pair_df[pair_df['apr'].isna()]

,pair1,pair2,tvl,apr,where,network,apy,type_of_agriculture
6585,REDUCE,,0,NaN,SUSHI,ETH,NaN,FARM
6620,SAFFRONCHEF,,0,NaN,SUSHI,ETH,NaN,FARM
6677,kmWETH,USDT,0,NaN,SUSHI,ETH,NaN,FARM
6678,kmWETH,USDC,0,NaN,SUSHI,ETH,NaN,FARM
6679,kmWETH,DAI,0,NaN,SUSHI,ETH,NaN,FARM
6680,kmWBTC,USDT,0,NaN,SUSHI,ETH,NaN,FARM
6681,kmWBTC,DAI,0,NaN,SUSHI,ETH,NaN,FARM
6682,kmWBTC,USDC,0,NaN,SUSHI,ETH,NaN,FARM
6683,kmLINK,USDT,0,NaN,SUSHI,ETH,NaN,FARM
6684,kmLINK,DAI,0,NaN,SUSHI,ETH,NaN,FARM


In [35]:
pair_df = pair_df.drop_duplicates()

In [36]:
single_piar_df = pair_df[pair_df['pair2'] == ""].copy()
double_coin_pair_df = pair_df[pair_df['pair2'] != ""].copy()
tmp_df = double_coin_pair_df.copy()

In [37]:
single_piar_df

,pair1,pair2,tvl,apr,where,network,apy,type_of_agriculture
0,UNI,,58975.79,133.38,https://penguinswap.eth.link,ETH,2.786223e+02,FARM
1,BTCx5-USDC-1Jun21-LP,,852291.43,437.23,https://app.compli.fi,ETH,7.619383e+03,FARM
2,BTCx5-USDC-1Jun21-UP,,17926.99,10393.52,https://app.compli.fi,ETH,5.239487e+41,FARM
3,BTCx5-USDC-1Jun21-DOWN,,54131.76,3442.06,https://app.compli.fi,ETH,1.928512e+16,FARM
4,ETHx5-USDC-1Jun21-LP,,3847.21,96862.25,https://app.compli.fi,ETH,2.512570e+207,FARM
...,...,...,...,...,...,...,...,...
6585,REDUCE,,0,NaN,SUSHI,ETH,NaN,FARM
6620,SAFFRONCHEF,,0,NaN,SUSHI,ETH,NaN,FARM
6736,MASTERCHEFDUMMY,,1.00,NaN,SUSHI,ETH,NaN,FARM
7239,Xi,,0.0,0.0,Sushi,ETH,NaN,POOL


In [38]:
double_coin_pair_df

,pair1,pair2,tvl,apr,where,network,apy,type_of_agriculture
13,COMFI,ETH,1075160.17,13304.62,https://app.compli.fi,ETH,1.847225e+51,FARM
23,USDC,QUICK,11634078.85,23.22,https://quickswap.exchange/#/quick,MATIC,2.612789e+01,FARM
24,ETH,QUICK,9124572.39,32.16,https://quickswap.exchange/#/quick,MATIC,3.791376e+01,FARM
26,USDC,DAI,47984534.68,4.92,https://quickswap.exchange/#/quick,MATIC,5.042693e+00,FARM
27,QUICK,TEL,2758231.03,77.65,https://quickswap.exchange/#/quick,MATIC,1.172058e+02,FARM
...,...,...,...,...,...,...,...,...
8290,ETH,0xMR,6.393288,24.30591528,Uniswap,ETH,NaN,POOL
8291,ETH,CHZ,40.560723,56.11707960000001,Uniswap,ETH,NaN,POOL
8292,ETH,POLS,4868.881122,4.3867717200000005,Uniswap,ETH,NaN,POOL
8293,ETH,89SA,3.998762,5.15366652,Sushi,ETH,NaN,POOL


In [39]:
tmp_df

,pair1,pair2,tvl,apr,where,network,apy,type_of_agriculture
13,COMFI,ETH,1075160.17,13304.62,https://app.compli.fi,ETH,1.847225e+51,FARM
23,USDC,QUICK,11634078.85,23.22,https://quickswap.exchange/#/quick,MATIC,2.612789e+01,FARM
24,ETH,QUICK,9124572.39,32.16,https://quickswap.exchange/#/quick,MATIC,3.791376e+01,FARM
26,USDC,DAI,47984534.68,4.92,https://quickswap.exchange/#/quick,MATIC,5.042693e+00,FARM
27,QUICK,TEL,2758231.03,77.65,https://quickswap.exchange/#/quick,MATIC,1.172058e+02,FARM
...,...,...,...,...,...,...,...,...
8290,ETH,0xMR,6.393288,24.30591528,Uniswap,ETH,NaN,POOL
8291,ETH,CHZ,40.560723,56.11707960000001,Uniswap,ETH,NaN,POOL
8292,ETH,POLS,4868.881122,4.3867717200000005,Uniswap,ETH,NaN,POOL
8293,ETH,89SA,3.998762,5.15366652,Sushi,ETH,NaN,POOL


In [40]:
tmp_df['tmp'] = tmp_df['pair1']
tmp_df['pair1'] = tmp_df['pair2']
tmp_df['pair2'] = tmp_df['tmp']
del tmp_df['tmp']

In [41]:
double_coin_pair_df.append(tmp_df).sort_values(['pair1', 'pair2']).reset_index(drop=True)

,pair1,pair2,tvl,apr,where,network,apy,type_of_agriculture
0,,sil,0.0,0.0,Sushi,ETH,NaN,POOL
1,,xSUSHI,0.0,0.0,Sushi,ETH,NaN,POOL
2,$BASED,ETH,12.640678,8.808390119999999,Uniswap,ETH,NaN,POOL
3,$BASED,SUSD,13.117297,5.72888256,Uniswap,ETH,NaN,POOL
4,$DG,ETH,687785.51,77.30,https://quickswap.exchange/#/quick,MATIC,116.448535,FARM
...,...,...,...,...,...,...,...,...
11913,zLOT,ETH,19.316181,2.16014808,Sushi,ETH,NaN,POOL
11914,zSUSHI,ZERO,92617.19,0.00,https://app.0.exchange,AVAX,0.000000,FARM
11915,zUNI,ZERO,14646.15,0.00,https://app.0.exchange,AVAX,0.000000,FARM
11916,zUSDC,ZERO,67883.09,0.00,https://app.0.exchange,AVAX,0.000000,FARM


In [42]:
summary = {
    "total_pairs": pair_df.shape[0],
"single_coin_pairs": single_piar_df.shape[0],
"double_coin_piars": double_coin_pair_df.shape[0],
"after_switching_coins_total_double_coin_pairs": double_coin_pair_df.shape[0] * 2,
"total_pairs_in_sheet": double_coin_pair_df.shape[0] * 2 + single_piar_df.shape[0]
}

In [43]:
summary

{'total_pairs': 8089,
 'single_coin_pairs': 2130,
 'double_coin_piars': 5959,
 'after_switching_coins_total_double_coin_pairs': 11918,
 'total_pairs_in_sheet': 14048}

In [44]:
pair_df = double_coin_pair_df.append(tmp_df).sort_values(['pair1', 'pair2']).reset_index(drop=True)
pair_df = single_piar_df.append(pair_df).sort_values(['pair1', 'pair2']).reset_index(drop=True)

In [45]:
pair_df

,pair1,pair2,tvl,apr,where,network,apy,type_of_agriculture
0,,sil,0.0,0.0,Sushi,ETH,NaN,POOL
1,,xSUSHI,0.0,0.0,Sushi,ETH,NaN,POOL
2,$BASED,ETH,12.640678,8.808390119999999,Uniswap,ETH,NaN,POOL
3,$BASED,SUSD,13.117297,5.72888256,Uniswap,ETH,NaN,POOL
4,$DG,ETH,687785.51,77.30,https://quickswap.exchange/#/quick,MATIC,116.448535,FARM
...,...,...,...,...,...,...,...,...
14043,zs-UST,,0.00,0,https://www.stabilize.finance/app/,ETH,0.000000,FARM
14044,zs-XUSD,,0.00,0,https://www.stabilize.finance/app/,ETH,0.000000,FARM
14045,zs-XUSD2,,0.00,0,https://www.stabilize.finance/app/,ETH,0.000000,FARM
14046,zs-tsBTC,,0.00,0,https://www.stabilize.finance/app/,ETH,0.000000,FARM


In [46]:
pair_df['dex'] = pair_df['where']
pair_df['date'] = date 
del pair_df['where']

In [47]:
pair_df.to_csv('yield_data_2021_06_14.csv')

In [48]:
pair_df['dex'].unique().tolist()

['Sushi',
 'Uniswap',
 'https://quickswap.exchange/#/quick',
 'https://strudel.finance',
 'https://swap.cometh.io/#/stake',
 'https://www.bao.finance',
 'https://caramelswap.finance',
 'https://absorber.finance',
 'https://icecreamswap.finance',
 'http://mangofarm.finance',
 'https://1inch.exchange/#/dao/farming',
 'https://babyswap.finance/ilo',
 'SUSHI',
 'https://app.swamp.finance',
 'https://ellipsis.finance',
 'https://frozenyogurt.finance',
 'https://nerve.fi',
 'https://autofarm.network',
 'https://beta.belt.fi',
 'https://blue.planetfinance.io',
 'https://glacierswap.org/farms/',
 'https://dollar.supra.finance',
 'https://polyvolve.finance',
 'https://polygaj.finance',
 'https://www.polycake.finance',
 'https://polyrangers.finance',
 'https://polyfi.io',
 'https://polylion.exchange',
 'https://www.orcinuslabs.ca',
 'https://polywolf.finance',
 'https://www.polygoat.finance',
 'https://www.moonwolf.io',
 'https://polycrab.finance',
 'https://polybear.exchange',
 'https://www.sme

In [49]:
pair_df[pair_df['pair1'] == 'ETH']

,pair1,pair2,tvl,apr,network,apy,type_of_agriculture,dex,date
5762,ETH,,11216054.78,30.09,MATIC,35.090676,FARM,https://polywhale.finance,2021-06-16
5763,ETH,,5968.09,390.02,MATIC,4740.046920,FARM,https://polyfox.finance,2021-06-16
5764,ETH,,1778.91,0.00,MATIC,0.000000,FARM,https://hawkdex.com,2021-06-16
5765,ETH,,19138898.37,0,MATIC,0.000000,FARM,https://polycat.finance,2021-06-16
5766,ETH,,774.45,71.77,MATIC,104.826955,FARM,https://www.polystarter.fi,2021-06-16
...,...,...,...,...,...,...,...,...,...
7852,ETH,zHEGIC,2.229828,1.31204424,ETH,NaN,POOL,Uniswap,2021-06-16
7853,ETH,zHEGIC,0.019139,0.0,ETH,NaN,POOL,Sushi,2021-06-16
7854,ETH,zLOT,0.550745,1.1481439199999999,ETH,NaN,POOL,Uniswap,2021-06-16
7855,ETH,zLOT,19.316181,2.16014808,ETH,NaN,POOL,Sushi,2021-06-16


In [50]:
pair_df['apr'].unique().tolist()

['0.0',
 '8.808390119999999',
 '5.72888256',
 '77.30',
 '5.9177233199999995',
 '107.88',
 '5.7588788399999995',
 '148.37',
 '3.30166308',
 '5.99870484',
 '263.81',
 '11.60412132',
 '3.10',
 '2.30547576',
 '24.30591528',
 '0.18127031999999998',
 '0.49014192',
 '0',
 '0.00',
 '230.98',
 '195.34',
 '36.38',
 '0.13',
 '21.187301399999996',
 '13.0237212',
 '0.38033039999999996',
 '107.61',
 nan,
 '1.2397871999999999',
 '0.27431196',
 '1.12040016',
 '1.348821',
 '0.9954314400000001',
 '0.0035435999999999996',
 '23.47',
 '15.77',
 '51.61',
 '5.42',
 '0.08908811999999999',
 '253.02',
 '0.07637052',
 '2.78224632',
 '5.15366652',
 '2.23',
 '7.85103612',
 '216.26',
 '125.39',
 '134.95',
 '152.46',
 '76.59',
 '325.36',
 '63.35',
 '350.34',
 '346.70',
 '159.28',
 '42.28',
 '70.18',
 '129.50',
 '58.14',
 '141.21',
 '423.63',
 '1129.67',
 '37.03',
 '117899.79',
 '64.26',
 '10445456.55',
 '126485.58',
 '15.85',
 '92.45',
 '30.75',
 '67.91',
 '924.67',
 '3867.94',
 '88.56',
 '0.82',
 '10.31',
 '123.66'

In [51]:
pair_df

,pair1,pair2,tvl,apr,network,apy,type_of_agriculture,dex,date
0,,sil,0.0,0.0,ETH,NaN,POOL,Sushi,2021-06-16
1,,xSUSHI,0.0,0.0,ETH,NaN,POOL,Sushi,2021-06-16
2,$BASED,ETH,12.640678,8.808390119999999,ETH,NaN,POOL,Uniswap,2021-06-16
3,$BASED,SUSD,13.117297,5.72888256,ETH,NaN,POOL,Uniswap,2021-06-16
4,$DG,ETH,687785.51,77.30,MATIC,116.448535,FARM,https://quickswap.exchange/#/quick,2021-06-16
...,...,...,...,...,...,...,...,...,...
14043,zs-UST,,0.00,0,ETH,0.000000,FARM,https://www.stabilize.finance/app/,2021-06-16
14044,zs-XUSD,,0.00,0,ETH,0.000000,FARM,https://www.stabilize.finance/app/,2021-06-16
14045,zs-XUSD2,,0.00,0,ETH,0.000000,FARM,https://www.stabilize.finance/app/,2021-06-16
14046,zs-tsBTC,,0.00,0,ETH,0.000000,FARM,https://www.stabilize.finance/app/,2021-06-16


In [52]:
pair_df[pair_df['pair1'] == 'ADA']

,pair1,pair2,tvl,apr,network,apy,type_of_agriculture,dex,date
126,ADA,,10350.40,0.00,BSC,0.000000,FARM,https://www.goosedefi.com,2021-06-16
127,ADA,,182381.64,0,BSC,0.000000,FARM,https://www.goosedefi.com,2021-06-16
128,ADA,,2704.61,0.00,BSC,0.000000,FARM,http://slime.finance,2021-06-16
129,ADA,,333221.90,28.56,BSC,33.041154,FARM,https://koaladefi.finance/,2021-06-16
130,ADA,,4724.61,306.47,BSC,2015.561358,FARM,https://brickchain.finance,2021-06-16
...,...,...,...,...,...,...,...,...,...
215,ADA,USDT,14470.53,263.40,BSC,1279.824525,FARM,https://autofarm.network,2021-06-16
216,ADA,USDT,107923.16,23.67,BSC,26.696374,FARM,https://app.swamp.finance,2021-06-16
217,ADA,USDT,"183,725",142.26,BSC,313.643732,FARM,https://babyswap.finance/ilo,2021-06-16
218,ADA,dADA,2346999.69,312.32,BSC,2141.910677,FARM,https://iron.finance/,2021-06-16


In [53]:
df['pair1'].unique().tolist()

['ALEPH',
 'APYS',
 'BOLE',
 'BOP',
 'BTC',
 'CHAD',
 'CHEEMS',
 'COPE',
 'ETH',
 'FIDA',
 'FROG',
 'FTR',
 'FTT',
 'KEEP',
 'KERMIT',
 'KIN',
 'LINK',
 'LUA',
 'MAPS',
 'MEDIA',
 'MER',
 'MOLA',
 'OXY',
 'RAY',
 'ROPE',
 'SAMO',
 'SBFC',
 'SDOGE',
 'SHBL',
 'SKEM',
 'SLIM',
 'SOL',
 'SOLAPE',
 'SOLPAD',
 'SRM',
 'STEP',
 'SUSHI',
 'TOMO',
 'TULIP',
 'TYNA',
 'unknown',
 'USDC',
 'USDT',
 'WOOF',
 'xCOPE',
 'YFI',
 'WBTC',
 'WHALE',
 'FRONT',
 'DGCL',
 'UNI',
 'YUNO',
 'DAI',
 'MIS',
 'QDAO',
 'BAO',
 'REN',
 'DATA',
 'sUSD',
 'CRO',
 'MIS3',
 'AXIE',
 'cDAI',
 'xSUSHI',
 'BNS',
 'SASHIMI',
 'MKR',
 'silv2',
 'FORTH',
 'WAN',
 'KP3R',
 'VXC',
 'OOKS',
 'ALBT',
 'KITTY',
 'jEUR',
 'sil',
 'EASY',
 'KAIJU',
 'LEND',
 'PROS',
 'MASK',
 'FOMC',
 'BNT',
 'OHM',
 'ARTH',
 'ANT',
 'DIA',
 'yveCRV',
 'CURA',
 'RUNE',
 'ibETH',
 'WOLF',
 'Xi',
 'SAKE',
 'RAI',
 'TUSD',
 'NAMI',
 'oneLINK',
 'XOR',
 'HEGIC',
 'FARM',
 'HBTC',
 'BUSD',
 'wCOINBASE',
 'Mars',
 'plDai',
 'TSD',
 'LON',
 'ELE',
 '1I

## Farm Risk

In [54]:
# # farms_risks_df = pd.read_excel("~/notebooks/vfat_tool_data/Farm_risk.xlsx", sheet_name='BSC')
# farm_risk_df = pd.read_csv('~/notebooks/vfat_tool_data/farm_risks_analysis.csv')

In [55]:
# farm_risk_df.columns = ['#', 'Indiv. Assesment', 'Audit Score', 'RugDoc', 'Farm Name', 'Farm Symbol', 'Farm Url', 'rugdoc_url', 'network']
# farm_risk_df


In [56]:
# pairs_farm_risk_df = pd.merge(pair_df, farm_risk_df, how='left', left_on=['dex', 'network'], right_on=['Farm Url', 'network'])
# pairs_farm_risk_df.to_csv('farm_risk_yield_data_2021_06_14.csv')
# pairs_farm_risk_df

In [57]:
# farms_df = pair_df[['dex', 'network']].drop_duplicates().reset_index(drop=True)

In [58]:
# farms_df

In [59]:
# new_farm_risk_df = pd.merge(farms_df, farm_risk_df, how='left', left_on=['dex', 'network'], right_on=['Farm Url', 'network'])
# new_farm_risk_df['Farm Url'] = new_farm_risk_df['dex']
# del new_farm_risk_df['dex']
# new_farm_risk_df = new_farm_risk_df[['#', 'Indiv. Assesment', 'Audit Score', 'RugDoc', 'Farm Name', 'Farm Symbol', 'Farm Url', 'rugdoc_url', 'network']]

# # new_farm_risk_df.columns = ['#', 'Indiv. Assesment', 'Audit Score', 'RugDoc', 'Farm Name', 'Farm Symbol', 'Farm Url', 'rugdoc_url', 'network']


In [60]:
# new_farm_risk_df.to_csv('Farm_risk_06_14.csv', index=False)

In [61]:
excel_file = '~/notebooks/vfat_tool_data/Farm_risk_20210616.xlsx'

In [62]:
bsc_risks_df = pd.read_excel(excel_file, sheet_name='BSC')
bsc_risks_df.columns = ['#', 'Indiv. Assesment', 'Audit Score', 'RugDoc', 'Farm Name', 'Farm Symbol', 'Farm Url', '1', '2', '3']
bsc_risks_df['rugdoc_url'] = bsc_risks_df['1'].fillna('') + bsc_risks_df['2'].fillna('') + bsc_risks_df['3'].fillna('')
del bsc_risks_df['1']
del bsc_risks_df['2']
del bsc_risks_df['3']
bsc_risks_df['network'] = 'BSC'
bsc_risks_df


,#,Indiv. Assesment,Audit Score,RugDoc,Farm Name,Farm Symbol,Farm Url,rugdoc_url,network
0,x,NaN,NaN,least,Pancake,CAKE,https://pancakeswap.finance,https://rugdoc.io/project/pancakeswap/,BSC
1,NaN,NaN,NaN,some,Apeswap,BANANA,https://apeswap.finance,https://rugdoc.io/project/apeswap/,BSC
2,NaN,NaN,NaN,NaN,Autofarm,AUTOFARM,https://autofarm.network,,BSC
3,x,9.0,NaN,NaN,Bake,BAKE,https://www.bakeryswap.org,,BSC
4,x,NaN,86.0,NaN,Brew,BREW,https://cafeswap.finance,https://www.certik.org/projects/cafeswap,BSC
...,...,...,...,...,...,...,...,...,...
324,NaN,NaN,NaN,low,VenomSwap,VENOM,https://www.venomswap.net,https://rugdoc.io/project/venom-swap/,BSC
325,NaN,NaN,NaN,low,Elephant,ELEPHANT,https://app.elephantswap.finance,https://rugdoc.io/project/elephant-swap/,BSC
326,No TLV,NaN,NaN,NaN,Bloodmoon,BLOODMOON,https://bloodmoon.werewolf.farm,,BSC
327,NaN,NaN,NaN,low,Blacksheep,BLACKSHEEP,https://www.blacksheepdefi.com,https://rugdoc.io/project/blacksheep-finance/,BSC


In [63]:
matic_risks_df = pd.read_excel(excel_file, sheet_name='MATIC')
matic_risks_df.columns = ['#', 'Indiv. Assesment', 'RugDoc', 'Farm Name', 'Farm Symbol', 'Farm Url', '1', '2']
matic_risks_df['Audit Score'] = np.nan
matic_risks_df['rugdoc_url'] = matic_risks_df['1'].fillna('') + matic_risks_df['2'].fillna('')
del matic_risks_df['1']
del matic_risks_df['2']
matic_risks_df['network'] = 'MATIC'
matic_risks_df

,#,Indiv. Assesment,RugDoc,Farm Name,Farm Symbol,Farm Url,Audit Score,rugdoc_url,network
0,x,low,NaN,Quickswap *,QUICK,https://quickswap.exchange/#/quick,NaN,,MATIC
1,x,some,NaN,Cometh,MUST,https://swap.cometh.io/#/stake,NaN,,MATIC
2,No TLV,NaN,NaN,Dark,DB,https://www.dark.build,NaN,,MATIC
3,NaN,NaN,NaN,Smartdex,NIOX,https://swap.smartdex.app,NaN,,MATIC
4,x,high,NaN,Elk,ELK,https://app.elk.finance,NaN,,MATIC
...,...,...,...,...,...,...,...,...,...
85,No TLV,NaN,NaN,PolyZero,ZERO,NaN,NaN,,MATIC
86,NaN,NaN,low,PolyMax,MAX,https://polymax.club,NaN,https://rugdoc.io/project/polymax/,MATIC
87,NaN,NaN,low,PolySnowFarm,SNOW,https://polysnow.farm,NaN,https://rugdoc.io/project/polysnow-farm/,MATIC
88,NaN,NaN,low,Polyyeld,YELD,https://polyyeld.finance,NaN,https://rugdoc.io/project/polyyeld-finance/,MATIC


In [64]:
avax_risks_df = pd.read_excel(excel_file, sheet_name='AVAX')
avax_risks_df.columns = ['#', 'Indiv. Assesment', 'RugDoc', 'Farm Name', 'Farm Symbol', 'Farm Url', '1', '2']
avax_risks_df['Audit Score'] = np.nan
avax_risks_df['rugdoc_url'] = avax_risks_df['1'].fillna('') + avax_risks_df['2'].fillna('')
del avax_risks_df['1']
del avax_risks_df['2']
avax_risks_df['network'] = 'AVAX'
avax_risks_df


,#,Indiv. Assesment,RugDoc,Farm Name,Farm Symbol,Farm Url,Audit Score,rugdoc_url,network
0,x,low,NaN,Pangolin,PNG,https://app.pangolin.exchange/#/png,NaN,,AVAX
1,x,low,NaN,Complus,COM,https://avax.complus.exchange,NaN,,AVAX
2,NaN,NaN,NaN,Zero,ZERO,https://app.0.exchange,NaN,,AVAX
3,NaN,NaN,NaN,Yeti,YTS,https://exchange.yetiswap.app,NaN,,AVAX
4,x,NaN,low,Pandaswap,BAMBOO,https://pandaswap.exchange,NaN,https://rugdoc.io/project/panda-swap-defi/,AVAX
5,x,some,NaN,Pefi,PEFI,https://penguinfinance.org,NaN,,AVAX
6,x,high,NaN,Elk *,ELK,https://elk.finance,NaN,,AVAX
7,No TLV,NaN,NaN,Snowball,SNOB,https://snowball.network/earn,NaN,,AVAX
8,x,some,NaN,Olive,OLIVE,https://avax.olive.cash,NaN,,AVAX
9,x,NaN,some,Lydia,LYD,https://www.lydia.finance,NaN,https://rugdoc.io/project/lydia-avax/,AVAX


In [65]:
fantom_risks_df = pd.read_excel(excel_file, sheet_name='FANTOM')
fantom_risks_df.columns = ['#', 'Indiv. Assesment', 'RugDoc', 'Farm Name', 'Farm Symbol', 'Farm Url', '1', '2']
fantom_risks_df['Audit Score'] = np.nan
fantom_risks_df['rugdoc_url'] = fantom_risks_df['1'].fillna('') + fantom_risks_df['2'].fillna('')
del fantom_risks_df['1']
del fantom_risks_df['2']
fantom_risks_df['network'] = 'FANTOM'
fantom_risks_df


,#,Indiv. Assesment,RugDoc,Farm Name,Farm Symbol,Farm Url,Audit Score,rugdoc_url,network
0,NaN,NaN,NaN,Popsicle,ICE,https://popsicle.finance,NaN,,FANTOM
1,x,NaN,low,Spooky,boo,https://spookyswap.finance,NaN,https://rugdoc.io/project/spookyswap/,FANTOM
2,x,NaN,some,Spirit,SPIRIT,https://app.spiritswap.finance,NaN,https://rugdoc.io/project/spiritswap/,FANTOM
3,x,NaN,low,Borg,SON,https://ftm.borgswap.exchange,NaN,https://rugdoc.io/project/borgswap-kling/,FANTOM
4,x,NaN,low,Hyperjump,ORI,https://hyperjump.fi,NaN,https://rugdoc.io/project/hyperjump-fi/,FANTOM
5,x,NaN,low,Ester,EST,https://ester.finance,NaN,https://rugdoc.io/project/ester-finance/,FANTOM
6,x,high,NaN,Froyo,FROYO,https://frozenyogurt.finance,NaN,,FANTOM
7,NaN,NaN,NaN,Ghost,GHOST,https://www.theghost.finance,NaN,,FANTOM
8,No TLV,NaN,low,Beefy,Various,https://fantom.beefy.finance,NaN,https://rugdoc.io/project/beefy-finance/,FANTOM
9,x,some,NaN,Liquid Driver,LQDR,https://www.liquiddriver.finance,NaN,,FANTOM


In [66]:
risk_df = bsc_risks_df.append(matic_risks_df)
risk_df = risk_df.append(avax_risks_df)
risk_df = risk_df.append(fantom_risks_df)
risk_df = risk_df[risk_df['rugdoc_url'] != '']
risk_df

,#,Indiv. Assesment,Audit Score,RugDoc,Farm Name,Farm Symbol,Farm Url,rugdoc_url,network
0,x,NaN,NaN,least,Pancake,CAKE,https://pancakeswap.finance,https://rugdoc.io/project/pancakeswap/,BSC
1,NaN,NaN,NaN,some,Apeswap,BANANA,https://apeswap.finance,https://rugdoc.io/project/apeswap/,BSC
4,x,NaN,86.0,NaN,Brew,BREW,https://cafeswap.finance,https://www.certik.org/projects/cafeswap,BSC
5,NaN,NaN,NaN,some,Donut,DONUT,https://donutdefi.com,https://rugdoc.io/project/donutswapp/,BSC
7,NaN,NaN,NaN,NaN,Goose,GOOSE,https://www.goosedefi.com,https://www.certik.org/projects/ramenswap,BSC
...,...,...,...,...,...,...,...,...,...
17,xx,NaN,NaN,high,Paintswap,BRUSH,https://paintswap.finance,https://rugdoc.io/project/paintswap/,FANTOM
18,xx,NaN,NaN,low,Tomb,Various,https://tomb.finance,https://rugdoc.io/project/tomb-finance/,FANTOM
19,NaN,NaN,NaN,high,Shadow,Various,https://shadowswap.app,https://rugdoc.io/project/shadowswap-finance/,FANTOM
20,NaN,NaN,NaN,low,Borgswap,KLING,https://klingftm.borgswap.exchange,https://rugdoc.io/project/borgswap-kling/,FANTOM


In [67]:
pairs_farm_risk_df = pd.merge(pair_df, risk_df, how='left', left_on=['dex', 'network'], right_on=['Farm Url', 'network'])
pairs_farm_risk_df.to_csv('farm_risk_yield_data_2021_06_14.csv')
pairs_farm_risk_df

,pair1,pair2,tvl,apr,network,apy,type_of_agriculture,dex,date,#,Indiv. Assesment,Audit Score,RugDoc,Farm Name,Farm Symbol,Farm Url,rugdoc_url
0,,sil,0.0,0.0,ETH,NaN,POOL,Sushi,2021-06-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,xSUSHI,0.0,0.0,ETH,NaN,POOL,Sushi,2021-06-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,$BASED,ETH,12.640678,8.808390119999999,ETH,NaN,POOL,Uniswap,2021-06-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,$BASED,SUSD,13.117297,5.72888256,ETH,NaN,POOL,Uniswap,2021-06-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,$DG,ETH,687785.51,77.30,MATIC,116.448535,FARM,https://quickswap.exchange/#/quick,2021-06-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14043,zs-UST,,0.00,0,ETH,0.000000,FARM,https://www.stabilize.finance/app/,2021-06-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14044,zs-XUSD,,0.00,0,ETH,0.000000,FARM,https://www.stabilize.finance/app/,2021-06-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14045,zs-XUSD2,,0.00,0,ETH,0.000000,FARM,https://www.stabilize.finance/app/,2021-06-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14046,zs-tsBTC,,0.00,0,ETH,0.000000,FARM,https://www.stabilize.finance/app/,2021-06-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
